In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !unzip /content/drive/MyDrive/Хакатон/train_dataset_dataset/data/drive-download-20240413T074335Z-001.zip -d /content/drive/MyDrive/Хакатон/train_dataset_dataset/data

In [ ]:
!pip install striprtf -q
!pip install pyrtf -q
!pip install pyth -q
!pip install pypiwin32 -q
!pip install docx2python -q
!pip install textract -q
!pip install docx2txt -q
!pip install pdfplumber python-docx -q
!pip install natasha -q

# LIBS

In [ ]:
import pandas as pd
import numpy as np
# -------------------------------------------
from striprtf.striprtf import rtf_to_text
import os
import pdfplumber
from docx import Document
import docx2txt
import textract
from docx2python import docx2python
import re

# -------------------------------------------
import nltk
nltk.download("stopwords")
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer, WordPunctTokenizer, word_tokenize

# -------------------------------------------
from sklearn.preprocessing import LabelEncoder

import spacy as sp
import sys

# !pip install ru_core_news_sm-3.5.0-py3-none-any.whl -q

import spacy
from spacy.tokens.doc import Doc

!python -m spacy download ru_core_news_lg -q

from sklearn.feature_extraction.text import TfidfVectorizer
# -------------------------------------------

from sklearn.metrics import f1_score, accuracy_score, classification_report

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 671.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 24.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Data

In [ ]:
# !unzip /content/drive/MyDrive/Хакатон/drive-download-20240413T081215Z-001.zip -d /content/drive/MyDrive/Хакатон/train_dataset_dataset

In [162]:
def read_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
        return text.strip()

def read_docx(file_path):
    doc = Document(file_path)
    paragraphs = []
    for para in doc.paragraphs:
        paragraphs.append(para.text)
    return '\n'.join(paragraphs)

def read_rtf(path):
    with open(path, 'rb') as f:
        content = f.read().decode('utf-8', errors='ignore')
        text = rtf_to_text(content)
        return [os.path.dirname(path).split("/")[-1], text, os.path.basename(path).split(".")[-1:]]

def is_word_file(file_path):
    with open(file_path, 'rb') as f:
        content = f.read()
        if content.startswith(b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1'):  # Магическое число для файлов DOC
            return True
        elif content.startswith(b'PK\x03\x04'):  # Магическое число для файлов DOCX
            return True
        else:
            return False

PATH = '/content/drive/MyDrive/Хакатон/train_dataset_dataset/data'
folders = os.listdir(PATH)
data = []
bugs = []

def read_file(file_path, label_folder, file_ext):
    try:
        if file_ext == 'rtf':
            return read_rtf(file_path)
        elif file_ext == 'pdf':
            return [label_folder, read_pdf(file_path), file_ext]
        elif file_ext in ['docx', 'doc']:
            if is_word_file(file_path):
                return [label_folder, read_docx(file_path), file_ext]
            else:
                print(f'Файл {file_path} не является поддерживаемым форматом Word')
                bugs.append(file_path)
        else:
            print(f'Файл {file_path} не является поддерживаемым форматом')
            bugs.append(file_path)
    except Exception as e:
        print(f'Файл {file} не был прочитан. Путь: {file_path}')
        print(e)
        bugs.append(file_path)

for label_folder in folders:
    if os.path.isdir(os.path.join(PATH, label_folder)):
        files = os.listdir(os.path.join(PATH, label_folder))
        for file in files:
            file_path = os.path.join(PATH, label_folder, file)
            file_ext = file.split(".")[-1]
            data.append(read_file(file_path, label_folder, file_ext))

In [170]:
data = pd.DataFrame(data, columns=['class', 'text', 'filename'])
data

,class,text,filename
0,contract offer,ДОГОВОР-ОФЕРТА (ПРИМЕР) 1. Общие положения 1.1...,[docx]
1,contract offer,Договор публичной оферты об оказании услуг\nОО...,[docx]
2,contract offer,Договор-оферта на оказание услуг 1. Общие поло...,[docx]
3,contract offer,ДОГОВОР-ОФЕРТА об оказании платных образовател...,[docx]
4,contract offer,"Договор-оферта интернет-магазина:\n\nООО ""ИНТЕ...",[rtf]
...,...,...,...
89,statute,"ТИПОВОЙ УСТАВ № 1, НА ОСНОВАНИИ КОТОРОГО ДЕЙСТ...",docx
90,statute,Утвержден\nприказом Минэкономразвития России\n...,docx
91,statute,УТВЕРЖДЕН:\n\nОбщим собранием Учредителей\nОбщ...,docx
92,statute,\t\tУТВЕРЖДЕН\n\t\t\t\t\t\t\t\t Решением ...,docx


In [171]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   class     94 non-null     object
 1   text      94 non-null     object
 2   filename  94 non-null     object
dtypes: object(3)
memory usage: 2.3+ KB


In [ ]:
data.isnull().sum()

In [172]:
df = pd.read_csv('/content/drive/MyDrive/Хакатон/train_dataset_dataset/data/sample.csv')
df

,class,text
0,arrangement,СОГЛАШЕНИЕ N 8\nо расторжении трудового догово...
1,arrangement,Соглашение о предоставлении опциона на заключе...
2,arrangement,Соглашение\nо реструктуризации задолженности\n...
3,arrangement,Дополнительное соглашение\r\nк договору купли-...
4,arrangement,Соглашение\nо расторжении договора об оказании...
...,...,...
496,bill,Счет № 5 от 01 октября 2020 г.\r\n\r\nПоставщи...
497,bill,Счет на оплату № от 14 октября 2020 года\r\n\r...
498,bill,Счет №23 от 12.09.2024 г.\t\t...
499,bill,"""Огурец!"" (ИП Микрюков В.В.)\t\t\t\t\t\t\r\n\t..."


In [173]:
data_n = pd.concat([df, data.drop(columns = 'filename')], axis=0)

In [ ]:
data_n.to_csv('/content/drive/MyDrive/Хакатон/train_dataset_dataset/data/data.csv', index=False)

# Text Cleaning

In [ ]:
# data = pd.read_csv('/content/drive/MyDrive/Хакатон/train_dataset_dataset/data/data.csv')
# data.info()

In [174]:
nlp = sp.load('ru_core_news_lg')

def del_NER(text):

    doc = nlp(text)
    for ent in doc.ents:
        text = text.replace(ent.text, '')

    return text

In [175]:
def clean(doc):

    doc = doc.lower().strip()
    doc = doc.replace('\n', ' ')
    doc = doc.replace('\r', ' ')
    doc = doc.replace('\t', ' ')
    # # html tags
    doc = re.sub(r'<.*?>', '', doc)
    # Замена адресов электронной почты на "почта"
    doc = re.sub(r'[\w\.-]+@[\w\.-]+', 'почта', doc)
    # Замена упоминаний на "тэг"
    doc = re.sub(r'@[\w]+', 'тэг', doc)
    # Замена URL-адреса любого вида на "сайт"
    doc = re.sub(r'(https?://)?(www\.)?[\w\.-]+\.[a-zа-я]+', 'сайт', doc)
    # # Замена дат на "дата"
    doc = re.sub(r"(\d{1,2}[./-]\d{1,2}[./-]\d{2,4})|(\d{2,4}[./-]\d{1,2}[./-]\d{1,2})", "дата", doc)
    # Удаление знаков препинания и цифр
    doc = re.sub('[^а-яА-Я0-9nN] | ([nN]\b)', ' ', doc)
    doc = re.sub(r'\s+', ' ', doc)

    stop_words = set(stopwords.words('russian'))
    doc = word_tokenize(doc)
    # Фильтрация стоп-слов
    doc = [word for word in doc if word not in stop_words]

    # Стемминг
    stemmer = SnowballStemmer("russian")
    doc = ' '.join([stemmer.stem(token) for token in doc])

    return doc

    # # Лемматизация
    # morph = pymorphy2.MorphAnalyzer()
    # lemmatized = [morph.parse(word)[0].normal_form for word in filtered_tokens]
    # print(lemmatized)

In [176]:
clean_data = data.copy()

clean_data['text'] = data['text'].apply(lambda x: clean(del_NER(x)))

In [ ]:
clean_data.to_csv('/content/drive/MyDrive/Хакатон/train_dataset_dataset/data/data_clean.csv', index = False)

# TRAIN

In [177]:
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

# Масштабирование данных
tfidf = TfidfVectorizer()
data_vector=tfidf.fit_transform(data['text']).toarray()

# Кодирование меток классов
# label_encoder = LabelEncoder()
# y_encoded = label_encoder.fit_transform(y)

# X_train, X_test, y_train, y_test = train_test_split(clean_data['text_vectors'], clean_data['class'], test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(data_vector, clean_data['class'], test_size=0.2, random_state=42)

# Gaussian Naive Bayes

In [178]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)

y_pred_train = gnb.predict(X_train)
y_pred_test = gnb.predict(X_test)
print("\nTraining Accuracy score:",f1_score(y_train, y_pred_train, average='micro'))
print("Testing Accuracy score:",f1_score(y_test, y_pred_test, average='micro'))

# Training Accuracy score: 1.0
# Testing Accuracy score: 0.8272727272727273


Training Accuracy score: 1.0
Testing Accuracy score: 1.0


# Multinomial Naive Bayes

In [179]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)

y_pred_train = mnb.predict(X_train)
y_pred_test = mnb.predict(X_test)
print("\nTraining Accuracy score:",f1_score(y_train, y_pred_train, average='micro'))
print("Testing Accuracy score:",f1_score(y_test, y_pred_test, average='micro'))

# Training Accuracy score: 0.9680365296803652
# Testing Accuracy score: 0.8454545454545455


Training Accuracy score: 1.0
Testing Accuracy score: 1.0


# Logistic Regression Classifier

In [180]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)
print("\nTraining Accuracy score:",f1_score(y_train, y_pred_train, average='micro'))
print("Testing Accuracy score:",f1_score(y_test, y_pred_test, average='micro'))

# Training Accuracy score: 0.9954337899543378
# Testing Accuracy score: 0.9363636363636364


Training Accuracy score: 1.0
Testing Accuracy score: 1.0


# Support Vector Machine

In [181]:
svc =  LinearSVC(class_weight='balanced')
svc.fit(X_train, y_train)

y_pred_train = svc.predict(X_train)
y_pred_test = svc.predict(X_test)
print("\nTraining Accuracy score:",f1_score(y_train, y_pred_train, average='micro'))
print("Testing Accuracy score:",f1_score(y_test, y_pred_test, average='micro'))

# Training Accuracy score: 1.0
# Testing Accuracy score: 0.990909090909091


Training Accuracy score: 1.0
Testing Accuracy score: 1.0


# Decision Tree

In [182]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

y_pred_train = dt.predict(X_train)
y_pred_test = dt.predict(X_test)
print("\nTraining Accuracy score:",f1_score(y_train, y_pred_train, average='micro'))
print("Testing Accuracy score:",f1_score(y_test, y_pred_test, average='micro'))

# Training Accuracy score: 1.0
# Testing Accuracy score: 0.8818181818181818


Training Accuracy score: 1.0
Testing Accuracy score: 1.0


# Ensembling

In [183]:
from sklearn.ensemble import VotingClassifier

classifiers = [('Decision Tree', dt),
               ('Logistic Regression', lr),
                ('Naive Bayes', gnb)
              ]
vc = VotingClassifier(estimators=classifiers)
# Fit 'vc' to the traing set and predict test set Labels
vc.fit(X_train, y_train)
y_pred_train=vc.predict(X_train)
y_pred_test = vc.predict(X_test)
print("Training Accuracy score:",f1_score(y_train, y_pred_train, average='micro'))
print("Testing Accuracy score:",f1_score(y_test, y_pred_test, average='micro'))

# Training Accuracy score: 1.0
# Testing Accuracy score: 0.9454545454545454

Training Accuracy score: 1.0
Testing Accuracy score: 1.0


# MLPClassifier

In [184]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(random_state=42)
mlp.fit(X_train, y_train)

y_pred_train = mlp.predict(X_train)
y_pred_test = mlp.predict(X_test)
print("\nTraining Accuracy score:",f1_score(y_train, y_pred_train, average='micro'))
print("Testing Accuracy score:",f1_score(y_test, y_pred_test, average='micro'))


Training Accuracy score: 1.0
Testing Accuracy score: 1.0


# LightGBM

In [185]:
import lightgbm as lgb

lgbm = lgb.LGBMClassifier(random_state=42, verbose=-1)
lgbm.fit(X_train, y_train)

y_pred_train = lgbm.predict(X_train)
y_pred_test = lgbm.predict(X_test)
print("\nTraining Accuracy score:",f1_score(y_train, y_pred_train, average='micro'))
print("Testing Accuracy score:",f1_score(y_test, y_pred_test, average='micro'))


Training Accuracy score: 1.0
Testing Accuracy score: 1.0


# Тестирование

In [166]:
q = read_docx('/content/schet-na-oplaty.docx')
q

'СЧЕТ  НА  ОПЛАТУ № (здесь порядковый номер) от (дата)\nПокупатель: наименование покупателя, его ИНН, адрес, телефон.\nУсловия поставки:  (например – самовывоз, обязательное уведомление об оплате, тогда резервирование товара на складе)\nобразец заполнения  платежного  поручения\n\n\n(Здесь приводятся сведения по Вашему желанию, обычно наименование фирмы (ИП, фамилия, имя, отчество, реквизиты  свидетельства  о  государственной регистрации  индивидуального  предпринимателя), адрес, телефоны, e-mail, иногда время действия счета и т.д.)\n\n(Далее имеет смысл перечислить товары или услуги по образцу накладной  и внизу обязательно указать: без налога (НДС), все это выглядеть может так) \n\n\nБез налога (НДС)\n\nВыписал (предприниматель):                                           (Фамилия, инициалы)\n\nБухгалтер (предприниматель):                                         (Фамилия, инициалы)\n\nРуководитель (предприниматель):                                   (Фамилия, инициалы)\nМ. П. '

In [186]:
q = clean(del_NER(q))

In [168]:
q

'порядков номер ( дат покупател наименован покупател ин адрес телефон услов поставк ( например самовывоз обязательн уведомлен оплат резервирован товар склад образец заполнен платежн поручен ( привод сведен ваш желан обычн наименован фирм ( фамил им отчеств реквизит свидетельств государствен регистрац индивидуальн предпринимател ) адрес телефон врем действ счет сайт . ( дал имеет смысл перечисл товар услуг образц накладн вниз обязательн указа налог ( ндс ) эт выглядет налог ( ндс ( предпринимател ) ( фамил инициал ( предпринимател ) ( фамил инициал руководител ( предпринимател ) ( фамил инициал )'

In [187]:
q_vec = tfidf.transform([q]).toarray()

print(f'GaussianNB          {gnb.predict(q_vec)}')
print(f'MultinomialNB       {mnb.predict(q_vec)}')
print(f'LogisticRegression  {lr.predict(q_vec)}')
print(f'Vector              {svc.predict(q_vec)}')
print(f'Decision Tree       {dt.predict(q_vec)}')
print(f'Voting Classifier   {vc.predict(q_vec)}')
print(f'MLPClassifier       {mlp.predict(q_vec)}')

GaussianNB          ['contract offer']
MultinomialNB       ['contract offer']
LogisticRegression  ['contract offer']
Vector              ['contract offer']
Decision Tree       ['contract offer']
Voting Classifier   ['contract offer']
MLPClassifier       ['contract offer']
